In [1]:
from pathlib import Path
import os
import sys
from dotenv import load_dotenv


def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / 'app').exists():
            return p
        if (p / 'backend' / 'app').exists():
            return p / 'backend'
    raise FileNotFoundError('Could not find repo root with app/')


ROOT = find_repo_root(Path.cwd().resolve())
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / '.env', override=True)
load_dotenv(ROOT / '.env.local', override=True)

# 캐시 비활성화 during notebook run
os.environ['RAG_RETRIEVE_CACHE'] = '0'
os.environ['RAG_CARD_CACHE'] = '0'
os.environ['RAG_DOC_SUMMARY_CACHE'] = '0'

print('ROOT:', ROOT)
print('DB:', os.getenv('DB_HOST'), os.getenv('DB_PORT'), os.getenv('DB_NAME'))
print('RETRIEVE_CACHE:', os.getenv('RAG_RETRIEVE_CACHE'), os.getenv('RAG_RETRIEVE_CACHE_TTL'))


ROOT: /Users/user/Desktop/rag_mid/backend
DB: localhost 5433 callact_db
RETRIEVE_CACHE: 0 None


In [2]:
import importlib
from app.rag.retriever import retriever, db as retriever_db, rank as retriever_rank, terms as retriever_terms
from app.rag.router import router
from app.rag.pipeline import retrieve as pipeline_retrieve
from app.rag.common import doc_source_filters
from app.llm import card_generator
from app.rag.pipeline import RAGConfig, run_rag

modules_to_reload = [
    retriever_terms,
    router,
    retriever_db,
    retriever_rank,
    pipeline_retrieve,
    doc_source_filters,
    card_generator,
    retriever,
]
for module in modules_to_reload:
    importlib.reload(module)

print('✅ Modules reloaded - card_products text_search restored')

✅ Modules reloaded - card_products text_search restored


In [3]:
import os
import sys
from pathlib import Path
root = Path().resolve()
while root != root.parent and not (root / 'backend').exists():
    root = root.parent
backend_root = root / 'backend'
if str(backend_root) not in sys.path:
    sys.path.insert(0, str(backend_root))
os.environ['RAG_LOG_RETRIEVER_DEBUG'] = '1'

from app.rag.tests.test_suite import TESTS, run_tests

await run_tests(TESTS, show_all=False)


[retriever] table=card_products fetch_ms=0.0 cand_added=11 total_cand=11 break_hit=False
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=94.1 rows=0
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=36.6 rows=8
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=162.8 rows=1
[retriever] source=guide_all fetch_ms=0.0 cand_added=8 total_cand=19 break_hit=False
[pipeline_retrieve] retrieve_ms=453.6 doc_count=4 route=card_usage
[cards] llm_input_chars=1073 ctx_chars=556 doc_ids=['narasarang_faq_005', 'narasarang_faq_006'] doc_chars=[283, 273]
[rag] route=700.0ms retrieve=453.7ms cards=2529.1ms post=0.1ms total=3682.9ms docs=4 route=card_usage
[retriever] table=card_products fetch_ms=0.0 cand_added=8 total_cand=8 break_hit=False
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_ms=127.3 rows=4
[retriever_db] table=service_guide_documents mode=text submode=trgm_like exec_m

In [4]:
# import json
# import sys
# from pathlib import Path
# root = Path().resolve()
# while root != root.parent and not (root / 'backend').exists():
#     root = root.parent
# backend_root = root / 'backend'
# if str(backend_root) not in sys.path:
#     sys.path.insert(0, str(backend_root))

# from app.rag.pipeline import RAGConfig, run_rag

# async def run_manual_rag():
#     query = input('질문을 입력하세요 (Enter로 종료): ').strip()
#     if not query:
#         print('입력된 질문이 없습니다. 종료합니다.')
#         return
#     cfg = RAGConfig(top_k=4)
#     response = await run_rag(query, config=cfg)
#     print('response JSON:')
#     print(json.dumps(response, ensure_ascii=False, indent=2))
# await run_manual_rag()


In [5]:
# import json
# import os
# import sys
# from pathlib import Path
# root = Path().resolve()
# while root != root.parent and not (root / 'backend').exists():
#     root = root.parent
# backend_root = root / 'backend'
# if str(backend_root) not in sys.path:
#     sys.path.insert(0, str(backend_root))
# os.environ['RAG_LOG_RETRIEVER_DEBUG'] = '1'
# from app.rag.pipeline import RAGConfig, run_rag

# async def run_manual_rag():
#     query = input('질문을 입력하세요 (Enter로 종료): ').strip()
#     if not query:
#         print('입력된 질문이 없습니다. 종료합니다.')
#         return
#     cfg = RAGConfig(top_k=4)
#     response = await run_rag(query, config=cfg)
#     print('response JSON:')
#     print(json.dumps(response, ensure_ascii=False, indent=2))
# await run_manual_rag()
